In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download("nltk")
nltk.download("punkt")

[nltk_data] Error loading nltk: Package 'nltk' not found in index
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df=pd.read_csv("/kaggle/input/healthcare-providers-data/Healthcare Providers.csv")

In [3]:
df["HCPCS Description"][0]

'Initial hospital inpatient care, typically 70 minutes per day'

In [4]:
df.iloc[:,10:]

,City of the Provider,Zip Code of the Provider,State Code of the Provider,Country Code of the Provider,Provider Type,Medicare Participation Indicator,Place of Service,HCPCS Code,HCPCS Description,HCPCS Drug Indicator,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount
0,SAINT LOUIS,631041004.0,MO,US,Internal Medicine,Y,F,99223,"Initial hospital inpatient care, typically 70 ...",N,27,24,27,200.58777778,305.21111111,157.26222222,160.90888889
1,FAYETTEVILLE,283043815.0,NC,US,Obstetrics & Gynecology,Y,O,G0202,"Screening mammography, bilateral (2-view study...",N,175,175,175,123.73,548.8,118.83,135.31525714
2,NORTH HAVEN,64732343.0,CT,US,Podiatry,Y,O,99348,"Established patient home visit, typically 25 m...",N,32,13,32,90.65,155,64.4396875,60.5959375
3,KANSAS CITY,641183998.0,MO,US,Internal Medicine,Y,O,81002,"Urinalysis, manual test",N,20,18,20,3.5,5,3.43,3.43
4,JUPITER,334585700.0,FL,US,Internal Medicine,Y,O,96372,Injection beneath the skin or into muscle for ...,N,33,24,31,26.52,40,19.539393939,19.057575758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,WILMINGTON,604811236.0,IL,US,Physical Therapist in Private Practice,Y,O,97162,"Evaluation of physical therapy, typically 30 m...",N,20,20,20,85.3725,214,60.7255,62.2485
99996,REDMOND,977561502.0,OR,US,Nurse Practitioner,Y,O,99213,Established patient office or other outpatient...,N,136,107,136,61.27,144.05147059,30.006176471,37.040220588
99997,SAINT LOUIS,631312322.0,MO,US,Cardiology,Y,F,93320,"Doppler ultrasound study of heart blood flow, ...",N,11,11,11,17.98,109.54545455,14.09,14.62
99998,LARCHMONT,105383500.0,NY,US,Internal Medicine,Y,O,G0008,Administration of influenza virus vaccine,N,12,12,12,30.54,65,29.93,25.32


In [5]:
one_hot=OneHotEncoder(sparse_output=False)
one=one_hot.fit_transform(df[["State Code of the Provider","Country Code of the Provider","HCPCS Drug Indicator","Last Name/Organization Name of the Provider","First Name of the Provider","Middle Initial of the Provider","Credentials of the Provider","Gender of the Provider","Entity Type of the Provider","Street Address 1 of the Provider","Street Address 2 of the Provider","City of the Provider","Provider Type","Medicare Participation Indicator","Place of Service"]])
one=pd.DataFrame(one,columns=one_hot.get_feature_names_out())


In [6]:
one

,State Code of the Provider_AA,State Code of the Provider_AE,State Code of the Provider_AK,State Code of the Provider_AL,State Code of the Provider_AP,State Code of the Provider_AR,State Code of the Provider_AZ,State Code of the Provider_CA,State Code of the Provider_CO,State Code of the Provider_CT,...,Provider Type_Sports Medicine,Provider Type_Surgical Oncology,Provider Type_Thoracic Surgery,Provider Type_Undefined Physician type,Provider Type_Urology,Provider Type_Vascular Surgery,Medicare Participation Indicator_N,Medicare Participation Indicator_Y,Place of Service_F,Place of Service_O
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
99996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
99997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
99998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [7]:
def text_preprocessing(text):
    data=text.lower()
    data_re=re.sub(r'\W=',' ',data)
    data_tokenize=word_tokenize(data_re)
    tokens=[word for word in data_tokenize if word not in stopwords.words("english")]
    return " ".join(tokens)

In [8]:
def text_vectorizer(text):
    vectorizer=TfidfVectorizer()
    vect=vectorizer.fit_transform(text)
    vect_pd=pd.DataFrame(vect.toarray(),columns=vectorizer.get_feature_names_out())
    return vect_pd

In [9]:
df["HCPCS Description"]=df["HCPCS Description"].apply(text_preprocessing)

In [10]:
# df

In [11]:
dd=text_vectorizer(df["HCPCS Description"])

In [12]:
concaty=pd.concat([df,dd],axis=1).drop(columns=["HCPCS Code","State Code of the Provider","Country Code of the Provider","HCPCS Drug Indicator","HCPCS Description","Last Name/Organization Name of the Provider","First Name of the Provider","Middle Initial of the Provider","Credentials of the Provider","Gender of the Provider","Entity Type of the Provider","Street Address 1 of the Provider","Street Address 2 of the Provider","City of the Provider","Provider Type","Medicare Participation Indicator","Place of Service"])


In [13]:
concaty

,index,National Provider Identifier,Zip Code of the Provider,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount,...,xi,xplus,yard,year,years,yeast,young,zinc,zoledronic,zoster
0,8774979,1891106191,631041004.0,27,24,27,200.58777778,305.21111111,157.26222222,160.90888889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3354385,1346202256,283043815.0,175,175,175,123.73,548.8,118.83,135.31525714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3001884,1306820956,64732343.0,32,13,32,90.65,155,64.4396875,60.5959375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7594822,1770523540,641183998.0,20,18,20,3.5,5,3.43,3.43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,746159,1073627758,334585700.0,33,24,31,26.52,40,19.539393939,19.057575758,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3837311,1386938868,604811236.0,20,20,20,85.3725,214,60.7255,62.2485,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99996,2079360,1215091327,977561502.0,136,107,136,61.27,144.05147059,30.006176471,37.040220588,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99997,8927965,1902868185,631312322.0,11,11,11,17.98,109.54545455,14.09,14.62,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99998,8854571,1891941183,105383500.0,12,12,12,30.54,65,29.93,25.32,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# # Deep learning AutoEncoder

In [14]:
from keras.layers import Dense,Input,Dropout,BatchNormalization
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [15]:
# early_stopping=EarlyStopping(
# monitor='val_loss',
# patience=5,
# verbose=1)

In [28]:
inputs=Input(shape=(2380,))
D=Dense(300,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
D=Dense(200,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dense(100,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dropout(0.2)(D)
D=Dense(100,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dense(200,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dense(300,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
outputs=Dense(2380,activation="sigmoid")(D)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [21]:
concaty_cleaned = concaty.replace(',', '', regex=True).astype(float)


In [22]:
concaty_cleaned

,index,National Provider Identifier,Zip Code of the Provider,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount,...,xi,xplus,yard,year,years,yeast,young,zinc,zoledronic,zoster
0,8774979.0,1.891106e+09,631041004.0,27.0,24.0,27.0,200.587778,305.211111,157.262222,160.908889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3354385.0,1.346202e+09,283043815.0,175.0,175.0,175.0,123.730000,548.800000,118.830000,135.315257,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3001884.0,1.306821e+09,64732343.0,32.0,13.0,32.0,90.650000,155.000000,64.439688,60.595937,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7594822.0,1.770524e+09,641183998.0,20.0,18.0,20.0,3.500000,5.000000,3.430000,3.430000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,746159.0,1.073628e+09,334585700.0,33.0,24.0,31.0,26.520000,40.000000,19.539394,19.057576,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3837311.0,1.386939e+09,604811236.0,20.0,20.0,20.0,85.372500,214.000000,60.725500,62.248500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99996,2079360.0,1.215091e+09,977561502.0,136.0,107.0,136.0,61.270000,144.051471,30.006176,37.040221,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99997,8927965.0,1.902868e+09,631312322.0,11.0,11.0,11.0,17.980000,109.545455,14.090000,14.620000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99998,8854571.0,1.891941e+09,105383500.0,12.0,12.0,12.0,30.540000,65.000000,29.930000,25.320000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
model.fit(concaty_cleaned,concaty_cleaned,epochs=10,batch_size=32)

Epoch 1/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 47s 14ms/step - accuracy: 0.0412 - loss: nan
Epoch 2/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 44s 14ms/step - accuracy: 0.0000e+00 - loss: nan
Epoch 3/10
1595/3125 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - accuracy: 0.0000e+00 - loss: nan